# Relatório EP DSDI

* A seguir vamos fazer um relátório explicando o que foi feito e por que foi feito do trabalho de DSID;
* Vamos usar o jupyter notebook para facilitar a relação entre código e texto e melhorar o entedimento;

## Código e Funções Utilizadas

* O serviço implementado consiste em um conjunto de métodos matemáticos que podem ser usados em vários tipos de aplicações diferentes. Esse métodos são:
1. void emptyRequest(): apenas para testar o tempo de resposta de um método sem parâmetros e retorno void;
2. double geometricVolume(double[3]): recebe os valores das dimensões de um paralelepípedo e retorna o valor do seu volume;
3. double geometricSurface(double[3]): recebe os valores das dimensões de um paralelepípedo e retorna o valor da sua área superficial;
4. double[] secondDegreeSolve(string): recebe um polinômio de segundo grau em formato de string e retorna os valores das suas raízes;
5. string[] thirdDegreeSolve(string): recebe um polinômio de terceiro grau em formato de string e retorna os valores das suas raízes (em formato de string por causa das raízes complexas);
6. string polynomeDerivative(string): recebe um polinômio qualquer em formato de string e retorna a sua derivada também em formato de string.
* Todos esses métodos foram implementados pelo grupo, com exceção do thirdDegreeSolve que foi retirado de https://www.blogcyberini.com/2017/09/algoritmo-equacao-terceiro-grau.html.
* Assim como descrito no trabalho a primeira ferramenta utilizada foi o gRPC, já a segunda foi utilizando a ferramenta jakarta, ambas feitas em java para não ter diferenças entre linguagens;


### gRPC

#### Definição do Protobuf

```proto
syntax = "proto3";
package proto;

option java_multiple_files = true;
option java_package = "generated";
option java_outer_classname = "Proto";

service  user {
  rpc EmptyReq(Empty) returns (Empty);

  rpc GeometricVolume(Solid) returns (Volume);
  rpc GeometricSurface(Solid) returns (Area);

  rpc SecondDegreeSolve(Polynome) returns (SecondDegreeRoot);
  rpc ThirdDegreeSolve(Polynome) returns (ThirdDegreeRoot);

  rpc PolynomeDerivative(Polynome) returns (Polynome);
}

message Solid {
  double height = 1;
  double length = 2;
  double width = 3;
}

message Area {
  double area = 1;
}

message Volume {
  double volume = 1;
}

message IntValue {
  int32 value = 1;
}

message Polynome {
  string polynome = 1;
}

message SecondDegreeRoot {
  double x1 = 1;
  double x2 = 2;
}

message ThirdDegreeRoot {
  string x1 = 1;
  string x2 = 2;
  string x3 = 3;
}

message Empty{

}
```

#### Servidor

* AppServer.java

```java
import io.grpc.ServerBuilder;
import io.grpc.Server;
import java.util.logging.Logger;
import java.io.IOException;

import service.Implementation;


public class AppServer {
    private static final Logger logger = Logger.getLogger(AppServer.class.getName());

    public static void main(String args[]) throws IOException, InterruptedException {

        try {
            Server server = ServerBuilder.forPort(50052).addService(new Implementation()).build();

            server.start();

            logger.info("Server started on " + server.getPort());

            server.awaitTermination();
        } catch (IOException e) {
            System.out.println("Error: " + e);
        } catch (InterruptedException e) {
            System.out.println("Error: " + e);
        }
    }

}

```

* Implementação do stub

```java
package service;


import java.util.Arrays;
import java.util.List;
import java.util.ArrayList;
import java.lang.Math;
import generated.userGrpc;
import generated.Area;
import generated.Empty;
import generated.IntValue;
import generated.Polynome;
import generated.Solid;
import generated.Volume;
import generated.ThirdDegreeRoot;
import generated.SecondDegreeRoot;

import io.grpc.stub.StreamObserver;

public class Implementation extends userGrpc.userImplBase {

    // Implementacao da logica

    @Override
    public void emptyReq(Empty request, StreamObserver<Empty> response) { // Parametros e retorno vazio
        System.out.println("EMPTY REQUEST");
        response.onCompleted();
    }

    @Override
    public void geometricVolume(Solid solid, StreamObserver<Volume> response) { // Calcula volume de um solido

        double volume = solid.getHeight() * solid.getLength() * solid.getWidth();

        Volume.Builder result = Volume.newBuilder();
        result.setVolume(volume);

        response.onNext(result.build());
        response.onCompleted();
    }

    @Override
    public void geometricSurface(Solid solid, StreamObserver<Area> response) { // Calcula area superficial de um solido
        double height = solid.getHeight();
        double width = solid.getWidth();
        double length = solid.getLength();

        double surf = (height * width * 2) + (height * length * 2) + (width * length * 2);

        Area.Builder result = Area.newBuilder();
        result.setArea(surf);

        response.onNext(result.build());
        response.onCompleted();
    }

    @Override
    public void secondDegreeSolve(Polynome polynome, StreamObserver<SecondDegreeRoot> response) { // Resolve equacoes de segundo grau

        String str = polynome.getPolynome();
        List<Integer> coeff = GetCoefficients(str, true);

        int a = coeff.get(0);
        int b = 0; int c = 0;
        if(coeff.size() >= 2)
            b = coeff.get(1);
        if(coeff.size() >= 3)
            c = coeff.get(2);

        double delta = b*b - 4 * a * c;
        double x1 = (-1 * b + Math.sqrt(delta)) / (2 * a);
        double x2 = (-1 * b - Math.sqrt(delta)) / (2 * a);

        response.onNext(SecondDegreeRoot.newBuilder().setX1(x1).setX2(x2).build());
        response.onCompleted();
    }

    @Override
    // Retirado de https://www.blogcyberini.com/2017/09/algoritmo-equacao-terceiro-grau.html
    public void thirdDegreeSolve(Polynome polynome, StreamObserver<ThirdDegreeRoot> response) { // Resolve equacoes de terceiro grau
        String str = polynome.getPolynome();
        List<Integer> coeff = GetCoefficients(str, true);

        int a = coeff.get(0);
        int b = 0; int c = 0; int d = 0;
        if(coeff.size() >= 2)
            b = coeff.get(1);
        if(coeff.size() >= 3)
            c = coeff.get(2);
        if(coeff.size() >= 4)
            d = coeff.get(3);

        System.out.println(a + " " + b + " " + c);

        double A = b / a;
        double B = c / a;
        double C = d / a;

        String[] roots = new String[3];

        double p = B - A * A / 3.0;
        double q = C + 2 * A * A * A / 27.0 - A * B / 3.0;

        double delta = q * q / 4.0 + p * p * p / 27.0;

        if (delta >= 0) {
            //primeira raiz
            double y1 = Math.cbrt(-q / 2.0 + Math.sqrt(delta)) + Math.cbrt(-q / 2.0 - Math.sqrt(delta));
            roots[0] = String.valueOf(y1 - A / 3.0);
            //discriminante secundário de uma equação do segundo grau
            double delta2 = -3.0 * y1 * y1 - 4.0 * p;
            if (delta2 >= 0) {
                roots[1] = String.valueOf((-y1 + Math.sqrt(delta2)) / 2.0 - A / 3.0);
                roots[2] = String.valueOf((-y1 - Math.sqrt(delta2)) / 2.0 - A / 3.0);
            } else {
                //raízes complexas
                double realPart = -y1 / 2.0;
                double imPart = Math.sqrt(Math.abs(delta2)) / 2.0;
                roots[1] = formatComplexResult(realPart - A / 3.0, imPart);
                roots[2] = formatComplexResult(realPart - A / 3.0, -imPart);
            }
        } else {
            //utiliza a fórmula de Euler para calcular as raízes
            double rho = Math.sqrt(q * q / 4.0 + Math.abs(delta));
            double theta = Math.acos(-q / (2.0 * rho));
            roots[0] = String.valueOf(2.0 * Math.cbrt(rho) * Math.cos(theta / 3.0) - A / 3.0);
            roots[1] = String.valueOf(2.0 * Math.cbrt(rho) * Math.cos((theta + 2.0 * Math.PI) / 3.0) - A / 3.0);
            roots[2] = String.valueOf(2.0 * Math.cbrt(rho) * Math.cos((theta + 4.0 * Math.PI) / 3.0) - A / 3.0);
        }

        response.onNext(ThirdDegreeRoot.newBuilder()
                .setX1(roots[0])
                .setX2(roots[1])
                .setX3(roots[2])
                .build()
        );

        response.onCompleted();
    }


    private static String formatComplexResult(double realPart, double imPart) {
        if (realPart == 0 && imPart == 0) {
            return "0";
        }
        String number = "";
        if (realPart != 0) {
            number += realPart;
            if (imPart > 0) {
                number += "+" + imPart + "i";
            } else if (imPart < 0) {
                number += imPart + "i";
            }
        } else {
            number += imPart + "i";
        }
        return number;
    }

    @Override
    public void polynomeDerivative(Polynome polynome, StreamObserver<Polynome> response) { // Calcula derivada de polinomios

        String str = polynome.getPolynome();
        List<Integer> coeffs = GetCoefficients(str, false);

        List<Integer> newCoeffs = new ArrayList<Integer>();
        for(int i = 1; i < coeffs.size(); i+=2) {
            newCoeffs.add(coeffs.get(i-1) * coeffs.get(i));
            newCoeffs.add(coeffs.get(i) - 1);
        }

        String result = new String();
        for(int i = 1; i < newCoeffs.size(); i+=2) {
            if(newCoeffs.get(i) == 0){
                if(newCoeffs.get(i-1) < 0){
                    result.concat(String.format("%s",
                            Integer.toString(newCoeffs.get(i-1))
                    ));
                } else {
                    result.concat(String.format("+%s",
                            Integer.toString(newCoeffs.get(i-1))
                    ));
                }
            } else {
                if(newCoeffs.get(i) < 0) {
                    result.concat(String.format("%s*x^%s",
                            Integer.toString(newCoeffs.get(i-1)),
                            Integer.toString(newCoeffs.get(i))
                    ));
                } else {
                    result.concat(String.format("+%s*x^%s",
                            Integer.toString(newCoeffs.get(i-1)),
                            Integer.toString(newCoeffs.get(i))
                    ));
                }
            }
        }


        response.onNext(Polynome.newBuilder()
                .setPolynome(result)
                .build()
        );

        response.onCompleted();

    }


    private List<Integer> GetCoefficients(String eq, boolean withoutExp) {
        String result;
        result = eq.replaceAll("[^0-9\\-\\.]+", " ");
        result= result.replaceAll("-", " -");
        String[] coeffAndExp = result.split(" ");
        List<Integer> coeff = new ArrayList<Integer>();

        if(withoutExp) { // Retorna lista apenas com coeficientes
            if(coeffAndExp[0].isEmpty())
                coeff.add(1);
            else
                coeff.add(Integer.parseInt(coeffAndExp[0]));
            if(coeffAndExp.length >= 3)
                coeff.add(Integer.parseInt(coeffAndExp[2]));
            if(coeffAndExp.length >= 5)
                coeff.add(Integer.parseInt(coeffAndExp[4]));
            if(coeffAndExp.length >= 7)
                coeff.add(Integer.parseInt(coeffAndExp[6]));

            return coeff;
        }

        // Retorna lista com os coeficientes e os expoentes
        for(String str : coeffAndExp) {
            if(str.isEmpty())
                coeff.add(1);
            else
                coeff.add(Integer.parseInt(str));
        }
        return coeff;

    }


}

```

#### Client

* Client.java
  
```java
import generated.userGrpc;
import generated.userGrpc.userBlockingStub;
import io.grpc.ManagedChannel;
import io.grpc.ManagedChannelBuilder;

import implementation.RequestCaller;
//
public class Client {

    public static void main(String[] args) {

        ManagedChannel channel = ManagedChannelBuilder.forAddress("localhost",50052).usePlaintext().build();

        userBlockingStub userStub = userGrpc.newBlockingStub(channel);

        RequestCaller caller = new RequestCaller(userStub);
        caller.start();

    }

}

```

* RequestCaller

```java
package implementation;

import generated.userGrpc;
import generated.userGrpc.userBlockingStub;

import generated.Empty;
import generated.Solid;
import generated.Volume;
import generated.Area;
import generated.Polynome;
import generated.SecondDegreeRoot;
import generated.ThirdDegreeRoot;

import java.util.Scanner;
import java.util.List;
import java.util.ArrayList;

public class RequestCaller {

    private final userBlockingStub stub;

    public RequestCaller(userBlockingStub stub) {
        this.stub = stub;
    }

    public void start() {
        Scanner sc = new Scanner(System.in);

        int procedure;
        double h;
        double l;
        double w;
        String pol;
        List<Double> roots;
        List<String> thirdRoots;
        long callTime; long responseTime; long elapsedTime;
        do {
            procedure = sc.nextInt();

            switch (procedure) {
                case 1: // emptyReq
                    callTime = System.nanoTime();
                    this.emptyReq();
                    responseTime = System.nanoTime();
                    elapsedTime = responseTime - callTime;
                    System.out.println("Response delay: " + elapsedTime);
                    break;
                case 2: // Calculo de volume
                    System.out.println("Enter solid height, length and width:");
                    h = sc.nextInt();
                    l = sc.nextInt();
                    w = sc.nextInt();

                    callTime = System.nanoTime();

                    double volume = this.geometricVolume(h, l, w);
                    responseTime = System.nanoTime();
                    System.out.println(String.format("Volume = %f", volume));

                    elapsedTime = responseTime - callTime;
                    System.out.println("Response delay: " + (double)elapsedTime / 1000000 + " ms");
                    break;
                case 3: // Calculo de superficie
                    System.out.println("Enter solid height, length and width:");
                    h = sc.nextInt();
                    l = sc.nextInt();
                    w = sc.nextInt();

                    callTime = System.nanoTime();

                    double area = this.geometricSurface(h, l, w);
                    responseTime = System.nanoTime();
                    System.out.println(String.format("Surface area = %f", area));

                    elapsedTime = responseTime - callTime;
                    System.out.println("Response delay: " + (double)elapsedTime / 1000000 + " ms");
                    break;
                case 4: // Resolucao equacao de segundo grau
                    System.out.println("Enter polynome:");
                    pol = sc.next();
                    System.out.println(pol);

                    callTime = System.nanoTime();

                    roots = this.secondDegreeSolve(pol);
                    responseTime = System.nanoTime();
                    System.out.println(String.format("x1 = %f | x2 = %f", roots.get(0), roots.get(1)));

                    elapsedTime = responseTime - callTime;
                    System.out.println("Response delay: " + (double)elapsedTime / 1000000 + " ms");
                    break;
                case 5: // Resolucao equacao de terceiro grau
                    System.out.println("Enter polynome:");
                    pol = sc.next();

                    callTime = System.nanoTime();

                    thirdRoots = this.thirdDegreeSolve(pol);
                    responseTime = System.nanoTime();
                    System.out.println(String.format("x1 = %s | x2 = %s | x3 = %s",
                            thirdRoots.get(0), thirdRoots.get(1), thirdRoots.get(2)));

                    elapsedTime = responseTime - callTime;
                    System.out.println("Response delay: " + (double)elapsedTime / 1000000 + " ms");
                    break;
                case 6: // Calculo de derivada
                    System.out.println("Enter polynome:");
                    pol = sc.next();

                    callTime = System.nanoTime();

                    String derivative = this.polynomeDerivative(pol);
                    responseTime = System.nanoTime();
                    System.out.println(String.format("Derivative: %s", derivative));

                    elapsedTime = responseTime - callTime;
                    System.out.println("Response delay: " + (double)elapsedTime / 1000000 + " ms");
                    break;
            }

        } while (procedure > 0);

    }

    private void emptyReq() {
        Empty emp = Empty.newBuilder().build();
        Empty response = this.stub.emptyReq(emp);
    }

    private double geometricVolume(double height, double length, double width) {
        Solid solid = Solid.newBuilder()
                .setHeight(height)
                .setLength(length)
                .setWidth(width)
                .build();

        Volume result = this.stub.geometricVolume(solid);
        return result.getVolume();
    }

    private double geometricSurface(double height, double length, double width) {
        Solid solid = Solid.newBuilder()
                .setHeight(height)
                .setLength(length)
                .setWidth(width)
                .build();

        Area result = this.stub.geometricSurface(solid);
        return result.getArea();
    }

    private List<Double> secondDegreeSolve(String pol) {
        Polynome polynome = Polynome.newBuilder().setPolynome(pol).build();
        SecondDegreeRoot result = this.stub.secondDegreeSolve(polynome);
        List<Double> roots = new ArrayList<>();
        roots.add(result.getX1());
        roots.add(result.getX2());
        return roots;
    }

    private List<String> thirdDegreeSolve(String pol) {
        Polynome polynome = Polynome.newBuilder().setPolynome(pol).build();
        ThirdDegreeRoot result = this.stub.thirdDegreeSolve(polynome);
        List<String> roots = new ArrayList<>();
        roots.add(result.getX1());
        roots.add(result.getX2());
        roots.add(result.getX3());
        return roots;
    }

    private String polynomeDerivative(String pol) {
        Polynome polynome = Polynome.newBuilder().setPolynome(pol).build();
        Polynome result = this.stub.polynomeDerivative(polynome);

        return result.getPolynome();
    }

}

```


### Jax /  Jakarta

## Construção das váriavéis

* Para nossa análise escolhemos 3 principais váriaveis:
  1. Tempo de resposta total;
  2. Tempo de processamento no servidor e no cliente;
  3. Latência (Tempo de resposta - (tempo de processamento em ambos os lados));
* Dessa forma conseguiremos analisar a maior parte das diferenãs entre as duas ferramentas utilizadas;

## Relatório

In [3]:
import matplotlib
import numpy as np

In [4]:
dale = np.array([])

In [5]:
print(dale)

[]
